# Homework 2 - Market Basket Analysis

> **Data Mining Class, 2022, JBNU**

아래 문제의 skeleton 코드를 구현하고 질문에 답하세요. 기본적인 코드는 수정할 필요가 없으며 `Fill your code here` 부분을 완성하면 됩니다. **아래 학번과 이름을 반드시 적어주세요.**

* **학번**: 201911832
* **이름**: 서민석

## Submission Guidline (제출요령)
> 모든 코드를 완성하고 난뒤에는 각 cell을 순차적으로 수행하여 결과를 출력하고 상단 메뉴에서 `파일 => 다운로드 => .ipynb 다운로드`를 클릭해서 파일을 다운받습니다. **이때 파일 이름은 반드시 `HW2_이름_학번.ipynb`으로 변경해주셔야 합니다.** 다음 다운로드 받은 파일을 LMS에 제출해주시면 됩니다. 
* 파일명예시: HW2_고양이_202212345.ipynb

## Notes
* 숙제의 취지를 해하지 않는 선에서 다른 package나 API 활용 가능합니다 (예, apriori를 구해주는 다른 package/API는 사용하면 안됩니다). 

## Warning
* 본 자료는 전북대학교 컴퓨터공학과 데이터마이닝 수업에서 사용되는 자료이며 무단 전재 및 재배포, 복사를 엄격히 금지합니다.
* 본 코드를 포함한 모든 수업자료에는 저작권법 제25조에 의해 수업 목적으로 이용한 저작물이 포함되어 있으므로 본 수업자료를 외부에 공개, 게시, 공유하는 것을 금지합니다. 

## Overview

이번 과제에서는 Spark을 활용하여 basket (transaction) 데이터로부터 연관 규칙 (association rule)추출을 위한 A-Priori 알고리즘을 구현하고 추출된 규칙을 분석하는 것이 목표입니다. 

### Setup
아래 코드를 실행해서 과제에 필요한 package들을 설치해주세요. 

In [1]:
!pip install -q pyspark py4j --quiet

     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 200 kB 49.4 MB/s 
     |████████████████████████████████| 199 kB 71.0 MB/s 


### Data Preparation
Market basket 분석을 위하여 transaction (basket) 정보가 담긴 데이터를 다운로드 합니다. 아래 코드는 `baskets.txt` 파일을 다운로드합니다. 

In [2]:
!wget -nc -q https://raw.githubusercontent.com/jinhongjung/raw_data/master/assignment2/baskets.txt

파일의 내용을 살펴보기 위해서 다음의 명령어를 실행합니다.

In [3]:
!head baskets.txt

whole milk,pastry,salty snack
sausage,whole milk,semi-finished bread,yogurt
soda,pickled vegetables
canned beer,misc. beverages
sausage,hygiene articles
sausage,whole milk,rolls/buns
whole milk,soda
frankfurter,soda,whipped/sour cream
frankfurter,curd
beef,white bread


`baskets.txt`에서 한 줄은 하나의 transaction (basket)을 의미합니다. 각 줄은 구매한 상품들로 이루어져 있고 `,` (comma)로 구분됩니다. 예로 첫 번째 바구니에는 `whole milk`, `pastry`, `salty snack`의 상품이 있습니다. 

In [4]:
!wc -l baskets.txt

14963 baskets.txt


`baskets.txt`은 총 `14,963`의 lines으로 구성되어 있으며 하나의 line은 하나의 basket을 의미하므로 총 `14,963`개의 basket이 입력으로 주어집니다. 

### Program Implementation for A-Priori Algorithm

이제 본격적으로 pyspark를 이용하여 A-Priori 알고리즘을 구현할 단계입니다. 

#### Spark Initialization
먼저 SparkConext를 생성하여 spark를 초기화해줍니다. 

In [5]:
from pyspark import SparkContext

sc = SparkContext("local", "a-priori")

#### Data Load and Preprocessing

`textFile()`을 이용해서 basket 데이터를 읽어옵니다. 

In [6]:
data_path = './baskets.txt'
data_rdd = sc.textFile(data_path)
data_rdd.take(5)

['whole milk,pastry,salty snack',
 'sausage,whole milk,semi-finished bread,yogurt',
 'soda,pickled vegetables',
 'canned beer,misc. beverages',
 'sausage,hygiene articles']

`data_rdd`는 원본 파일의 각 줄 (line)을 하나의 엔트리로 리스트 형태로 가지고 있습니다. A-Priori 알고리즘의 입력 형태를 갖추기 위해서 다음의 전처리를 수행합니다.
- `names`: 고유 상품 (item)의 list로 각 원소는 상품의 이름 (string)입니다.
- `name2idx`: 상품의 이름이 주어지면 `names`에서 해당 이름의 index를 반환하는 dictionary입니다. 

In [7]:
names = data_rdd.flatMap(lambda x: x.split(',')) \
                .groupBy(lambda x: x) \
                .keys() \
                .collect()
name2idx = {name:i for (i, name) in enumerate(names)}

print(name2idx['herbs']) # 53
print(names[53]) #herbs
print('# of total items: {}'.format(len(names)))

53
herbs
# of total items: 167


`name2idx`를 이용하여 `data_rdd`의 각 상품을 index로 변환해줍니다. 아래 코드는 순서대로 `,`로 tokenizing을 수행하고, name (string)에서 index (integer)로 변환한 뒤 (list comprehension), 각 list를 정렬합니다. 

In [8]:
tran_rdd = data_rdd.map(lambda x: set(x.split(','))) \
                   .map(lambda x: [name2idx[name] for name in x]) \
                   .map(lambda x: sorted(x))

print('The first 5 baskets: ' + str(tran_rdd.take(5)))

The first 5 baskets: [[0, 1, 2], [0, 3, 4, 5], [6, 7], [8, 9], [3, 10]]


결과를 보면 `tran_rdd`의 첫 번째 바구니는 `[0, 1, 2]`로 0번, 1번, 2번 아이템으로 구성된 것을 의미합니다. 나머지 바구니도 같은 방식으로 해석하면 됩니다. 

#### Notations in A-Priori Algorithm
A-Priori 구현을 일관되고 간결하게 하기 위해서 다음의 조건과 기호를 확인합니다. 
* 하나의 itemset은 tuple로 표현되며 tuple내에서는 아이템 번호기준으로 오름차순 정렬되어 있어야 합니다. 
    - 예시: (0, 1, 2) 또는 (0, 1)과 같이 표현됩니다. (3, 1, 2)와 같이 표현되면 안됩니다. 
    - 아이템이 하나인 경우 (0,)과 같이 표현됩니다 (마지막에 `,`가 붙음)
* `k`: itemset의 아이템 개수를 의미합니다 (k-tuple). (0, 1, 2)면 k가 3입니다 (3-tuple).
* `Ls`: actual frequent tuples의 list입니다. 
    - `Ls[k]`는 dictionary로 k-tuple을 key로 가지고 value는 해당 tuple의 support를 가집니다. 즉, `Ls`는 `list`이고 각 원소는 `dict`입니다. 
* `Ck`: candidate k-tuples의 `set`을 의미합니다. 



#### Q1. The `generate_candidates` Function Implementation <font color='red'>[20 points]</font>

먼저 actual frequent tuples을 입력으로 받아서 candidate tuples을 생성하는 함수를 작성합니다. 구체적으로 `Ls`와 `k`를 입력으로 받아서 `Ck`를 반환해야 합니다. 아래 함수에서 `Fill your code here` 부분을 구현하세요. 
* `prev_tuples`는 `Ls[k-1]`에서 itemset들만 모아서 사전순으로 정렬한 list입니다. 
* 조합을 간결하게 하기 위해서 itertools의 combinations을 활용하세요.
* *Optional challenge*: 가독성을 해하지 않으면서 최대 5줄 이내로 채워보기! (이렇게 안해도 점수에는 지장없습니다).

In [9]:
from itertools import combinations as comb

def generate_candidates(Ls, k):
    '''
    Ls: list of dictionary of {k-tuple : support}
    k: size of itemset
    '''
    Ck = set() # set of candidate k-tuples
    prev_tuples = sorted(Ls[k-1].keys()) # list of actual (k-1)-tuples
    
    for i in comb(prev_tuples, 2):
        if i[0][:-1] == i[1][:-1] and i[0][-1] < i[1][-1]:
            c = i[0] + i[1][-1:]
            [Ck.add(c) for u in prev_tuples if u == c[1:]]
    
    return Ck

함수가 제대로 구현되었는지 아래 코드를 수행해서 테스트 하세요 (AssertionError가 발생하지 않아야 함).

In [10]:
Ls = list()
Ls.append({})   # add dummy dictionary to make index start from 1
Ls.append({     # dict of 1-tuples (key) and their supports (value)
    (0,) : 10, 
    (1,) : 10,
    (2,) : 10,
    (3,) : 10
})
Ls.append({     # dict of 2-tuples (key) and their supports (value)
    (0, 1) : 5,
    (0, 2) : 5,
    (1, 2) : 5,
    (0, 3) : 5
})
Ls.append({     # dict of 3-tuples (key) and their supports (value)
    (0, 1, 2) : 3
})

ans_C2 = set([(0, 1), (1, 2), (1, 3), (2, 3), (0, 3), (0, 2)]) # do not modify
C2 = generate_candidates(Ls, 2)
print("C2: " + str(C2))
assert C2 == ans_C2

ans_C3 = set([(0, 1, 2)]) # do not modify
C3 = generate_candidates(Ls, 3)
print("C3: " + str(C3))
assert C3 == ans_C3

ans_C4 = set() # do not modify
C4 = generate_candidates(Ls, 4)
print("C4: " + str(C4))
assert C4 == ans_C4

C2: {(0, 1), (1, 2), (1, 3), (2, 3), (0, 3), (0, 2)}
C3: {(0, 1, 2)}
C4: set()


#### Q2. The `apriori` Function Implementation <font color='red'>[30 points]</font>
위에서 구현한 `generate_candidates` 함수와 pyspark API를 활용해서 `apriori`함수를 구현하세요. `apriori`함수는 입력으로 `tran_rdd`와 `supp_threshold`를 입력으로 받습니다 (`supp_threshold`는 0에서 1사이의 값을 가집니다). 
* transaction data로 부터 support를 count하고 filtering을 할 때는 pyspark RDD transformation API를 활용해야 합니다. 
* filtering을 할 때는 아래의 `s`를 활용해서 수행해주세요. 
* `Ls`에서는 위에서 설명한 포맷의 `dict` type의 원소만 있어야 합니다 (그래야 return에서 제대로 dictionary union 됩니다).

In [11]:
from collections import ChainMap

def apriori(tran_rdd, n, supp_threshold=0.01):
    '''
    tran_rdd: transaction rdd
    n: number of total baskets
    supp_threshold: support threshold between 0 and 1
    '''
    Ls = list() 
    Ls.append({}) # initialized with a dummy dictionary
    s = supp_threshold * n # support threshold

    ## count and filter
    rdd1 = tran_rdd.map(lambda x: comb(x, 1)) \
                   .flatMap(lambda x: x) \
                   .map(lambda x: (x, 1)) \
                   .reduceByKey(lambda a, b: a + b) \
                   .filter(lambda x: x[1] >= s)

    Ls.append({}) # Ls[1]
    for x in rdd1.collect():
        Ls[1][x[0]] = x[1]

    k = 2

    while len(Ls[k-1]) != 0:
        Ck = generate_candidates(Ls, k)
        Dt = tran_rdd.map(lambda x: comb(x, k)) \
                     .flatMap(lambda x: list(x)) \
                     .map(lambda x: (x, 1)) \
                     .reduceByKey(lambda a, b: a + b) \
                     .filter(lambda x: x[1] >= s)
        Ls.append({})
        for c in Dt.collect():
            Ls[k][c[0]] = c[1]
        k += 1

    return dict(ChainMap(*Ls)) # union of list of dictionaries

#### Q3. Number of Frequent Itemsets <font color='red'>[5 points]</font>
`supp_threshold`가 `0.001`일 때 `apriori` 함수에 의해 추출된 itemsets은 총 몇 개인가요? 아래 코드를 수행해서 확인해보세요. 

In [12]:
n = tran_rdd.count() # total number of baskets
L = apriori(tran_rdd, n, supp_threshold=0.001)
print('# of frequent itemsets: {}'.format(len(L)))

# of frequent itemsets: 750


#### Q4. Association Rule Extractions <font color='red'>[15 points]</font>
위에서 구한 결과 `L`로 부터 association rule을 추출하는 함수를 구현하세요. 이때 `conf_threshold`를 기준으로 filtering이 되어야 하고 결과값은 pandas table (dataframe)에 저장이 되어야 합니다. 
* Table은 `antecedents`, `consequents`, `confidence`, `lift`의 column으로 구성되며 하나의 row는 하나의 association rule을 의미합니다. 
    - Rule은 `antecedents => consequents`의 형태로 표현되며 이때 confidence와 lift를 계산해 같이 저장합니다. 
* Table에 하나의 row를 추가하기 위해서 아래의 코드를 활용합니다. `antecedents` (or `consequents`)은 아이템 이름 (string)의 set이어야 합니다. socres는 실수로 저장합니다.
```python
rules = rules.append({"antecedents": a set of item names,
                        "consequents": a set of item names,
                        "confidence": confidence score,
                        "lift": lift score}, ignore_index = True)
```

In [25]:
import pandas as pd
    
def extract_association_reuls(L, n, names, conf_threshold=0.1):
    '''
    L: dictionary of k-tuple (key) and its support (value)
    n: number of total baskets
    names: list of item names
    conf_threshold: confidence threshold between 0 and 1
    '''
    rules = pd.DataFrame(columns=['antecedents', 'consequents', 'confidence', 'lift'])

    for i in L:
        antecedentsSet = set()
        antecedentsSet = [antecedentsSet | set(comb(i, k)) for k in range(1, len(i))]
        
        for antecedents in antecedentsSet:
            for a in antecedents:
                consequents = sorted(list(set(i) - set(a)))
                confidence = L[tuple(i)] / L[tuple(a)] # support(a -> c)/ support(a)
                p = L[tuple(consequents)] / n # support(j) / n
                lift = confidence / p
                if confidence >= conf_threshold:
                    rules = rules.append({'antecedents': {names[x] for x in list(a)},
                                          'consequents': {names[x] for x in list(consequents)},
                                          'confidence': confidence,
                                          'lift': lift}, ignore_index = True)
        
    
    return rules

아래 코드를 실행해서 table 결과를 확인합니다. 구글 colab에는 pandas DataFrame에 대해 몇몇 interactive operations들을 제공합니다 (table 옆의 magicstick을 클릭해보세요).

In [26]:
rules = extract_association_reuls(L, n, names, conf_threshold=0.05)
rules

,antecedents,consequents,confidence,lift
0,"{whole milk, sausage}",{yogurt},0.164179,1.911760
1,"{yogurt, whole milk}",{sausage},0.131737,2.182917
2,"{yogurt, sausage}",{whole milk},0.255814,1.619866
3,"{rolls/buns, whole milk}",{sausage},0.081340,1.347825
4,"{whole milk, sausage}",{rolls/buns},0.126866,1.153275
...,...,...,...,...
445,{beverages},{other vegetables},0.104839,0.858622
446,{misc. beverages},{rolls/buns},0.084746,0.770383
447,{long life bakery product},{sausage},0.059701,0.989273
448,{cream cheese },{yogurt},0.067797,0.789448


#### Q5. Result Analysis <font color='red'>[30 points]</font>
* **Top-5 confident rules** [10 points]: confidence score를 기준으로 상위 5개의 rule을 출력하는 코드를 작성하세요 (pandas table을 출력하세요).

In [15]:
cs_sort_df = rules.sort_values(by=['confidence'], ascending=False).head(5)
cs_sort_df

,antecedents,consequents,confidence,lift
2,"{yogurt, sausage}",{whole milk},0.255814,1.619866
5,"{rolls/buns, sausage}",{whole milk},0.212500,1.345594
20,"{soda, sausage}",{whole milk},0.179775,1.138374
31,{semi-finished bread},{whole milk},0.176056,1.114825
11,"{rolls/buns, yogurt}",{whole milk},0.170940,1.082428


* **Top-5 interesting rules** [10 points]: lift score를 기준으로 상위 5개의 rule을 출력하는 코드를 작성하세요 (pandas table을 출력하세요).

In [16]:
ls_sort_df = rules.sort_values(by=['lift'], ascending=False).head(5)
ls_sort_df

,antecedents,consequents,confidence,lift
1,"{yogurt, whole milk}",{sausage},0.131737,2.182917
0,"{whole milk, sausage}",{yogurt},0.164179,1.911760
269,{specialty chocolate},{citrus fruit},0.087866,1.653762
2,"{yogurt, sausage}",{whole milk},0.255814,1.619866
423,{flour},{tropical fruit},0.109589,1.617141


* **Interpretation** [10 points]: 위 두 결과를 비교했을 때 뚜렷한 차이가 있나요? 차이가 있다면 어떠한 이유 때문에 차이가 발생하는 것 일까요?

```
confidence score로 정렬한 경우 support가 높은 consequents 순으로 정렬되는 경향이 있고, lift score의 경우는 그렇지 않았다.
support(consequents)가 높다면 confidence score가 높게 측정될 확률이 높지만 그것이 antecedents와 consequents의 연관성을 보장해주진 않기 때문이다.
```

### Terminate Spark
HW2 문제는 여기까지 입니다. 아래는 해당 SparkContext를 종료하는 코드입니다.

In [27]:
sc.stop()